# RBPの本数を計算するNotebook

In [1]:
import yaml
from pathlib import Path
import pandas as pd
from py3dbp import Packer, Bin, Item

In [2]:
def generate_result_df(box_names: list) -> pd.DataFrame:
      # check the result
  dict_bins = dict()

  for b in packer.bins:
      if b.items:
          s = pd.Series([b.items[i].name for i in range(len(b.items))])
          d = dict()
          for n in box_names:
              d[n] = s.str.contains(n).sum()
          dict_bins[b.name] = d

  return pd.DataFrame.from_dict(dict_bins, orient="index")


def add_bins(num=100):
    for i in range(num):
        packer.add_bin(Bin(
            f"rbp{i}", 
            rbp_width-rbp_margin, 
            rbp_height, 
            rbp_depth-rbp_margin, 
            rbp_max_weight))
            

def reset_packer():  # resetがうまくいかず、連続実行ができない
    packer.bins = []
    packer.items = []
    packer.unfit_items = []
    packer.total_items = 0

    add_bins()

In [3]:
path_root = Path.cwd().parent

In [4]:
# RBPのサイズを読み込み
with open(path_root/"config/config.yaml", "r") as s:
    config = yaml.safe_load(s)

rbp_width = config["rbp"]["width"]
rbp_height = config["rbp"]["height"]
rbp_depth = config["rbp"]["depth"]
rbp_margin = config["rbp"]["margin"]
rbp_max_weight = config["rbp"]["max_weight"]

In [5]:
# テストデータ読み込み
data = pd.read_csv(path_root/"data/test_data.csv")

In [6]:
# initialize packer
packer = Packer()

# add rbp to packer
add_bins()

patterns = [c for c in data.columns if "pattern" in c]
cols = [c for c in data.columns if c not in patterns]

In [7]:
# RBP本数の計算
"""
params: 
    Item(name, width, height, depth, weight)
"""
for p in patterns:
    df = data[cols+[p]]

    # add items for each size
    for s in df.size_name:
        df_size = df[df.size_name==s]
        for i in range(df_size[p].values[0]):
            packer.add_item(Item(
                f"{s}_{i}",
                int(df_size.width.values[0]), # packer doesn't accept np.int
                int(df_size.height.values[0]),
                int(df_size.depth.values[0]),
                int(df_size.weight.values[0])
            ))

    packer.pack(bigger_first=True, distribute_items=True)
    df_res = generate_result_df(df.size_name.values)

    # save as csv
    df_res.to_csv(path_root/f"results/results_{p}.csv")
    reset_packer()